In [1]:
import sys
sys.path.append('../')

import os
import json
import pandas as pd
from sentence_transformers import SentenceTransformer

# from read_data.utils import grab_dataset
from preprocessing import clean_and_standardize_text

/home/slwanna/miniconda3/envs/neural-feature-extraction/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
# Load Dataset
dataset_dir_path = "/home/slwanna/dataset-download-scripts/language_only_datasets/"
dataset_name = "rt1"

with open(f"{dataset_dir_path}metadata.json", "r", encoding="utf-8") as json_file:
        # The metadata file referenced here is contained in the repo:
        # dataset-download-scripts package hosted in the larger GitHub group.
        metadata_dict = json.load(json_file)

ds_path = metadata_dict[dataset_name]

# Begin text feature processing
df = pd.read_csv(dataset_dir_path + ds_path)

df = clean_and_standardize_text(df, nl_column="nl_instructions")

# Load Model
model_name = "all-mpnet-base-v2" # sbert
model = SentenceTransformer(model_name, trust_remote_code=True)

data = list(df["nl_instructions"])
# Grab embeddings
EMBEDDINGS = model.encode(data)

/home/slwanna/miniconda3/envs/neural-feature-extraction/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
# Save results
model_name = model_name.replace("/", "_")
results_dir = "results"
results_fp = results_dir + "/01_" + model_name + "_" + dataset_name

os.makedirs(results_dir, exist_ok=True)

results_dict = {
    "nl_command_exs": data,
    "embedding": EMBEDDINGS.tolist(),
}
results_df = pd.DataFrame(results_dict)
results_df.to_csv(f"{results_fp}.csv")